## Setup: Import Observability Modules

We'll use the modules we created in `src/observability/`.

In [1]:
import sys
from pathlib import Path

# Add src directory to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

# We import Prometheus first so we can clear old collectors before
# observability.metrics registers new ones (prevents duplicate errors)
from prometheus_client import REGISTRY
from contextlib import contextmanager

# Clear any existing metrics from previous runs (helpful when rerunning cells)
collector_map = getattr(REGISTRY, '_collector_to_names', None)
if collector_map:
    for collector in list(collector_map.keys()):
        try:
            REGISTRY.unregister(collector)
        except KeyError:
            pass

# Import our observability modules (safe after registry cleanup)
from observability.tracer import ContractAgentTracer
from observability.metrics import MetricsCollector, timed_operation
from observability.logger import ContractAgentLogger


print("✅ Observability modules imported successfully!")

✅ Observability modules imported successfully!


## Part 1: Structured Logging

Structured logging outputs JSON instead of plain text, making it:
- **Searchable** - Query by fields
- **Parseable** - Automated analysis
- **Contextual** - Rich metadata

In [2]:
import sys
from pathlib import Path

# Ensure src is on the path if setup cell was skipped
src_path = Path.cwd().parent / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import logger if it is missing from the namespace
try:
    ContractAgentLogger
except NameError:
    from observability.logger import ContractAgentLogger

import json
from datetime import datetime

# Initialize logger
logger = ContractAgentLogger("notebook_demo")

# Example 1: Basic logging
logger.info("Contract analysis started", extra={
    "request_id": "req-12345",
    "user_id": "user-001",
    "contract_type": "NDA"
})

# Example 2: Performance logging
logger.info("LLM call completed", extra={
    "duration_ms": 1234,
    "tokens_used": 450,
    "model": "gpt-4o-mini"
})

# Example 3: Error logging
logger.error("Classification failed", extra={
    "error_type": "ValidationError",
    "error_message": "Invalid contract format"
})

print("\n✅ Logged 3 structured events")
print("\nNote: In production, these would go to a log aggregation system like:")
print("  • Elasticsearch + Kibana")
print("  • Splunk")
print("  • Datadog")
print("  • CloudWatch Logs")

2025-11-29 11:14:38,773 - notebook_demo - INFO - Contract analysis started
2025-11-29 11:14:38,774 - notebook_demo - INFO - LLM call completed
2025-11-29 11:14:38,775 - notebook_demo - ERROR - Classification failed

✅ Logged 3 structured events

Note: In production, these would go to a log aggregation system like:
  • Elasticsearch + Kibana
  • Splunk
  • Datadog
  • CloudWatch Logs


### Log Context Management

Set request-wide context that automatically appears in all logs:

In [3]:
# Set context for the entire request
logger.set_request_context(
    request_id="req-demo-001",
    user_id="demo_user",
    session_id="session-abc123"
)

# All subsequent logs include this context automatically
logger.info("Step 1: PDF extraction")
logger.info("Step 2: Classification")
logger.info("Step 3: Analysis")

print("✅ Context automatically added to all log entries")

# Clear context when done
logger.clear_request_context()
print("✅ Context cleared")

2025-11-29 11:14:53,206 - notebook_demo - INFO - Step 1: PDF extraction
2025-11-29 11:14:53,207 - notebook_demo - INFO - Step 2: Classification
2025-11-29 11:14:53,207 - notebook_demo - INFO - Step 3: Analysis
✅ Context automatically added to all log entries
✅ Context cleared


## Part 2: Prometheus Metrics

Metrics track **what's happening** in your system over time.

### **Metric Types:**
1. **Counter** - Things that only go up (requests, errors)
2. **Gauge** - Values that go up/down (queue size, active users)
3. **Histogram** - Distribution of values (latency, size)
4. **Summary** - Like histogram with percentiles

In [4]:
# Initialize metrics collector
metrics = MetricsCollector()

# Simulate some contract analyses
print("Simulating contract analysis metrics...\n")

# Analysis 1: Successful NDA
metrics.record_request(contract_type="NDA", status="success")
metrics.record_llm_tokens(model="gpt-4o-mini", operation="classify", tokens=350)
metrics.record_duration(operation="classify", duration=1.2)
print("✅ Recorded NDA analysis (1.2s, 350 tokens)")

# Analysis 2: Successful SaaS
metrics.record_request(contract_type="SaaS", status="success")
metrics.record_llm_tokens(model="gpt-4o-mini", operation="analyze", tokens=1200)
metrics.record_duration(operation="analyze", duration=3.5)
print("✅ Recorded SaaS analysis (3.5s, 1200 tokens)")

# Analysis 3: Failed (error)
metrics.record_request(contract_type="Unknown", status="error")
metrics.record_duration(operation="classify", duration=0.5)
print("❌ Recorded failed analysis (0.5s)")

# Security events
metrics.record_pii_detection(entity_type="email", count=2)
metrics.record_pii_detection(entity_type="phone", count=1)
print("\n🔒 Recorded PII detections (2 emails, 1 phone)")

# Compliance checks
metrics.record_compliance_check(check_type="gdpr", result="pass")
metrics.record_compliance_check(check_type="data_retention", result="pass")
print("\n✅ Recorded compliance checks")

Simulating contract analysis metrics...

✅ Recorded NDA analysis (1.2s, 350 tokens)
✅ Recorded SaaS analysis (3.5s, 1200 tokens)
❌ Recorded failed analysis (0.5s)

🔒 Recorded PII detections (2 emails, 1 phone)

✅ Recorded compliance checks


### View Current Metrics

Prometheus exposes metrics in a specific text format:

In [5]:
from prometheus_client import generate_latest, REGISTRY

# Get current metrics in Prometheus format
metrics_output = generate_latest(REGISTRY).decode('utf-8')

# Show sample metrics
print("Sample Prometheus Metrics:\n")
print("=" * 80)

# Filter to show our custom metrics
for line in metrics_output.split('\n'):
    if 'contract_' in line or 'llm_' in line or 'pii_' in line:
        if not line.startswith('#'):
            print(line)

print("\n" + "=" * 80)
print("\nThese metrics would be scraped by Prometheus every 15 seconds")
print("and visualized in Grafana dashboards.")

Sample Prometheus Metrics:

contract_analysis_requests_total{contract_type="NDA",status="success"} 1.0
contract_analysis_requests_total{contract_type="SaaS",status="success"} 1.0
contract_analysis_requests_total{contract_type="Unknown",status="error"} 1.0
contract_analysis_requests_created{contract_type="NDA",status="success"} 1.7643951055561466e+09
contract_analysis_requests_created{contract_type="SaaS",status="success"} 1.7643951055561466e+09
contract_analysis_requests_created{contract_type="Unknown",status="error"} 1.7643951055561466e+09
contract_analysis_duration_seconds_bucket{complexity="unknown",contract_type="unknown",le="0.1"} 0.0
contract_analysis_duration_seconds_bucket{complexity="unknown",contract_type="unknown",le="0.5"} 1.0
contract_analysis_duration_seconds_bucket{complexity="unknown",contract_type="unknown",le="1.0"} 1.0
contract_analysis_duration_seconds_bucket{complexity="unknown",contract_type="unknown",le="2.0"} 2.0
contract_analysis_duration_seconds_bucket{complex

### Timed Operations Decorator

Automatically measure function execution time:

In [6]:
import time

@timed_operation("pdf_extraction")
def extract_pdf_simulation():
    """Simulate PDF extraction."""
    time.sleep(0.5)  # Simulate work
    return "Extracted text"

@timed_operation("contract_classification")
def classify_simulation():
    """Simulate classification."""
    time.sleep(0.8)  # Simulate LLM call
    return "NDA"

# Run timed operations
print("Running timed operations...\n")

result1 = extract_pdf_simulation()
print(f"✅ PDF extracted: {result1}")

result2 = classify_simulation()
print(f"✅ Classified as: {result2}")

print("\n⏱️  Duration metrics automatically recorded!")

Running timed operations...

✅ PDF extracted: Extracted text
✅ Classified as: NDA

⏱️  Duration metrics automatically recorded!


## Part 3: Distributed Tracing with OpenTelemetry

Traces show the **journey** of a request through your system.

### **Trace Anatomy:**
```
Trace (entire request)
├── Span: HTTP Request (parent)
│   ├── Span: PDF Extraction
│   ├── Span: Classification
│   │   └── Span: LLM Call
│   ├── Span: Analysis
│   │   └── Span: LLM Call
│   └── Span: Report Generation
└── Total: 5.2 seconds
```

In [7]:
# Initialize tracer
tracer = ContractAgentTracer(service_name="contract-analysis-notebook")

print("✅ OpenTelemetry tracer initialized")
print(f"   Service: contract-analysis-notebook")
print(f"   Exporting to: http://localhost:4318 (OTLP HTTP)")

✅ OpenTelemetry tracer initialized
   Service: contract-analysis-notebook
   Exporting to: http://localhost:4318 (OTLP HTTP)


### Create Traced Operations

Use the `trace_span` context manager to instrument your code:

In [8]:
import time
import random

def simulate_contract_analysis_with_tracing():
    """Simulate a full contract analysis with distributed tracing."""
    
    # Parent span for the entire operation
    with tracer.trace_span(
        "contract_analysis",
        attributes={
            "request.id": "req-trace-001",
            "user.id": "demo_user",
            "contract.type": "NDA"
        }
    ):
        print("🔍 Starting traced contract analysis...\n")
        
        # Step 1: PDF Extraction
        with tracer.trace_span(
            "pdf_extraction",
            attributes={"file.size": 52000, "file.pages": 3}
        ):
            time.sleep(0.3)
            print("  ✅ PDF extraction (300ms)")
        
        # Step 2: Security Validation
        with tracer.trace_span(
            "security_validation",
            attributes={"validator": "presidio"}
        ):
            time.sleep(0.2)
            print("  ✅ Security validation (200ms)")
        
        # Step 3: Classification
        with tracer.trace_span(
            "classification",
            attributes={
                "model": "gpt-4o-mini",
                "tokens.input": 350,
                "tokens.output": 50
            }
        ):
            time.sleep(0.8)
            print("  ✅ Classification (800ms, 400 tokens)")
        
        # Step 4: Detailed Analysis
        with tracer.trace_span(
            "detailed_analysis",
            attributes={
                "model": "gpt-4o-mini",
                "tokens.input": 1200,
                "tokens.output": 300
            }
        ):
            time.sleep(1.5)
            print("  ✅ Detailed analysis (1500ms, 1500 tokens)")
        
        # Step 5: Compliance Check
        with tracer.trace_span(
            "compliance_check",
            attributes={"checks": ["gdpr", "data_retention"]}
        ):
            time.sleep(0.4)
            print("  ✅ Compliance check (400ms)")
        
        # Step 6: Report Generation
        with tracer.trace_span(
            "report_generation",
            attributes={"format": "json"}
        ):
            time.sleep(0.2)
            print("  ✅ Report generation (200ms)")
        
        print("\n✅ Contract analysis complete!")

# Run the traced operation
simulate_contract_analysis_with_tracing()

print("\n📊 Trace exported to OpenTelemetry Collector")
print("   View in: Jaeger UI (http://localhost:16686) when Docker stack is running")

🔍 Starting traced contract analysis...

  ✅ PDF extraction (300ms)
  ✅ Security validation (200ms)
  ✅ Classification (800ms, 400 tokens)
  ✅ Detailed analysis (1500ms, 1500 tokens)
  ✅ Compliance check (400ms)
  ✅ Report generation (200ms)

✅ Contract analysis complete!

📊 Trace exported to OpenTelemetry Collector
   View in: Jaeger UI (http://localhost:16686) when Docker stack is running


### Trace with Custom Attributes

Add business context to your traces:

In [9]:
def analyze_with_rich_tracing(contract_type: str, has_pii: bool, complexity: str):
    """Analysis with rich trace attributes."""
    
    with tracer.trace_span(
        "contract_analysis",
        attributes={
            # Business context
            "contract.type": contract_type,
            "contract.complexity": complexity,
            "contract.has_pii": has_pii,
            
            # Technical context
            "service.version": "1.0.0",
            "environment": "production",
            
            # User context
            "user.id": "user-123",
            "user.org": "ACME Corp"
        }
    ) as span:
        time.sleep(0.5)
        
        # Add events to the span
        span.add_event("Classification completed", {
            "confidence": 0.95,
            "contract_type": contract_type
        })
        
        time.sleep(0.3)
        
        span.add_event("PII detected", {
            "entity_types": ["email", "phone"],
            "count": 3
        })
        
        return {"status": "success", "type": contract_type}

# Run with different contract types
print("Running analyses with rich tracing...\n")

result1 = analyze_with_rich_tracing("NDA", has_pii=True, complexity="Simple")
print(f"✅ {result1['type']} analyzed")

result2 = analyze_with_rich_tracing("SaaS", has_pii=False, complexity="Complex")
print(f"✅ {result2['type']} analyzed")

print("\n📊 Traces include business context and events!")

Running analyses with rich tracing...

✅ NDA analyzed
✅ SaaS analyzed

📊 Traces include business context and events!


## Part 4: Integrate Observability with LangGraph

Now let's add observability to our contract analysis agent from Notebook 1.

In [10]:
# Import from Notebook 1
from dotenv import load_dotenv
import os

load_dotenv(Path.cwd().parent / ".env", override=True)

# Import LangGraph components
import fitz
from typing import TypedDict, List, Optional, Dict, Any
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, END
import uuid

print("✅ LangGraph components imported")

✅ LangGraph components imported


In [11]:
# Import state definition from src/agent/state.py
from agent.state import ContractAnalysisState, create_initial_state

print("✅ ContractAnalysisState imported from src/agent/state.py")

✅ ContractAnalysisState imported from src/agent/state.py


### Observed Classification Node

Classification with full observability:

In [12]:
class ContractClassification(BaseModel):
    contract_type: str
    complexity: str
    confidence_score: float
    reasoning: str

def classify_contract_observed(state: ContractAnalysisState) -> ContractAnalysisState:
    """
    Classify contract with full observability instrumentation.
    """
    request_id = state['request_id']
    
    # Set logging context
    logger.set_request_context(
        request_id=request_id,
        user_id=state['user_id']
    )
    
    # Start trace span
    with tracer.trace_span(
        "classify_contract",
        attributes={
            "request.id": request_id,
            "user.id": state['user_id'],
            "text.length": len(state['contract_text'])
        }
    ) as span:
        try:
            logger.info("Starting contract classification")
            
            # Initialize LLM
            llm = ChatOpenAI(
                model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
                temperature=0
            )
            structured_llm = llm.with_structured_output(ContractClassification)
            
            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are an expert contract analyst. Classify the contract.
                
Contract Types: NDA, SaaS, Employment, Partnership, Unknown
Complexity: Simple, Moderate, Complex"""),
                ("user", "Classify this contract:\n\n{contract_text}")
            ])
            
            chain = prompt | structured_llm
            text_sample = state['contract_text'][:4000]
            
            # Measure LLM call
            with tracer.trace_span(
                "llm_call_classify",
                attributes={"model": "gpt-4o-mini", "max_tokens": 500}
            ):
                result = chain.invoke({"contract_text": text_sample})
            
            # Record metrics
            metrics.record_request(
                contract_type=result.contract_type,
                status="success"
            )
            metrics.record_llm_tokens(
                model="gpt-4o-mini",
                operation="classify",
                tokens=400  # Approximate
            )
            
            # Update state
            state['contract_type'] = result.contract_type
            state['complexity'] = result.complexity
            state['confidence_score'] = result.confidence_score
            
            # Log success
            logger.info("Classification successful", extra={
                "contract_type": result.contract_type,
                "complexity": result.complexity,
                "confidence": result.confidence_score
            })
            
            # Add trace event
            span.add_event("classification_complete", {
                "type": result.contract_type,
                "confidence": result.confidence_score
            })
            
            print(f"  ✅ Classified as {result.contract_type} ({result.confidence_score:.2f})")
            
        except Exception as e:
            # Record error metrics
            metrics.record_request(contract_type="Unknown", status="error")
            
            # Log error
            logger.error("Classification failed", extra={
                "error": str(e),
                "error_type": type(e).__name__
            })
            
            # Record in span
            span.record_exception(e)
            span.set_status("error", str(e))
            
            state['errors'].append(f"Classification error: {str(e)}")
            state['contract_type'] = "Unknown"
            
        finally:
            logger.clear_request_context()
    
    return state

print("✅ Observed classification node defined")

✅ Observed classification node defined


### Test Observed Classification

Let's test with a sample contract:

In [13]:
# Sample contract text
sample_nda_text = """
NON-DISCLOSURE AGREEMENT

This Non-Disclosure Agreement ("Agreement") is entered into as of January 1, 2024,
by and between TechCorp Inc. ("Disclosing Party") and John Doe ("Receiving Party").

1. CONFIDENTIAL INFORMATION
For purposes of this Agreement, "Confidential Information" means all information
disclosed by Disclosing Party to Receiving Party.

2. OBLIGATIONS
Receiving Party agrees to:
a) Maintain confidentiality of all Confidential Information
b) Not disclose to any third parties
c) Use only for the Purpose described herein

3. TERM
This Agreement shall remain in effect for 2 years from the Effective Date.
"""

# Create test state using the helper function from src/agent/state.py
test_state = create_initial_state(
    contract_text=sample_nda_text,
    file_path="sample_nda.txt",
    user_id="demo_user"
)

print("Running observed classification...\n")
result = classify_contract_observed(test_state)

print(f"\nResults:")
print(f"   Type: {result['contract_type']}")
print(f"   Complexity: {result['complexity']}")
print(f"   Confidence: {result['confidence_score']:.2%}")
print(f"\nLogs, metrics, and traces recorded!")

Running observed classification...

2025-11-29 11:20:05,596 - notebook_demo - INFO - Starting contract classification
2025-11-29 11:20:08,549 - notebook_demo - INFO - Classification successful
  ✅ Classified as NDA (95.00)

Results:
   Type: NDA
   Complexity: Simple
   Confidence: 9500.00%

Logs, metrics, and traces recorded!


## Part 5: Metrics Dashboard (Simulated)

In production, Grafana would visualize these metrics. Let's simulate a dashboard view:

In [14]:
import random
from collections import defaultdict

# Simulate 100 contract analyses
print("📊 Simulating 100 contract analyses...\n")

contract_types = ["NDA", "SaaS", "Employment", "Partnership"]
stats = defaultdict(int)
total_tokens = 0
durations = []

for i in range(100):
    contract_type = random.choice(contract_types)
    success = random.random() > 0.05  # 95% success rate
    
    # Record metrics
    metrics.record_request(
        contract_type=contract_type,
        status="success" if success else "error"
    )
    
    if success:
        tokens = random.randint(800, 2000)
        duration = random.uniform(1.0, 4.0)
        
        metrics.record_llm_tokens(
            model="gpt-4o-mini",
            operation="analyze",
            tokens=tokens
        )
        metrics.record_duration(
            operation="analyze",
            duration=duration
        )
        
        stats[contract_type] += 1
        total_tokens += tokens
        durations.append(duration)

# Display dashboard-style summary
print("=" * 80)
print("                    CONTRACT ANALYSIS DASHBOARD")
print("=" * 80)

print("\n📈 REQUEST METRICS")
print(f"   Total Requests:     100")
print(f"   Success Rate:       {len(durations)}%")
print(f"   Error Rate:         {100-len(durations)}%")

print("\n📊 CONTRACT TYPE BREAKDOWN")
for ctype, count in sorted(stats.items(), key=lambda x: x[1], reverse=True):
    bar = "█" * (count // 2)
    print(f"   {ctype:15} {count:3} {bar}")

print("\n⏱️  PERFORMANCE METRICS")
print(f"   Avg Duration:       {sum(durations)/len(durations):.2f}s")
print(f"   Min Duration:       {min(durations):.2f}s")
print(f"   Max Duration:       {max(durations):.2f}s")
print(f"   P95 Duration:       {sorted(durations)[int(len(durations)*0.95)]:.2f}s")

print("\n🪙 TOKEN USAGE")
print(f"   Total Tokens:       {total_tokens:,}")
print(f"   Avg Tokens/Request: {total_tokens//len(durations):,}")
print(f"   Est. Cost (GPT-4o-mini): ${total_tokens * 0.00015 / 1000:.2f}")

print("\n" + "=" * 80)
print("\nℹ️  In production, this data appears in Grafana dashboards with:")
print("   • Time-series graphs")
print("   • Real-time alerts")
print("   • Custom panels")
print("   • SLA tracking")

📊 Simulating 100 contract analyses...

                    CONTRACT ANALYSIS DASHBOARD

📈 REQUEST METRICS
   Total Requests:     100
   Success Rate:       92%
   Error Rate:         8%

📊 CONTRACT TYPE BREAKDOWN
   Partnership      26 █████████████
   SaaS             24 ████████████
   Employment       23 ███████████
   NDA              19 █████████

⏱️  PERFORMANCE METRICS
   Avg Duration:       2.45s
   Min Duration:       1.04s
   Max Duration:       3.94s
   P95 Duration:       3.70s

🪙 TOKEN USAGE
   Total Tokens:       126,152
   Avg Tokens/Request: 1,371
   Est. Cost (GPT-4o-mini): $0.02


ℹ️  In production, this data appears in Grafana dashboards with:
   • Time-series graphs
   • Real-time alerts
   • Custom panels
   • SLA tracking


## Part 6: Starting the Observability Stack

To view metrics and traces in production tools, start the Docker stack:

In [15]:
print("""🐳 To start the full observability stack:

1. Open terminal and navigate to project root:
   cd c:\\000 - Zensar - AI For Leaders\\References\\zensar-day6\\enterprise-contract-agent

2. Start Docker services:
   docker-compose up -d

3. Access the dashboards:
   • Prometheus:  http://localhost:9090
   • Grafana:     http://localhost:3000 (admin/admin)
   • Jaeger:      http://localhost:16686 (traces)

4. In Grafana:
   - Add Prometheus datasource (http://prometheus:9090)
   - Import dashboard from dashboards/contract_agent.json

5. Run the agent and see metrics appear in real-time!

""")

print("✅ Instructions displayed above")

🐳 To start the full observability stack:

1. Open terminal and navigate to project root:
   cd c:\000 - Zensar - AI For Leaders\References\zensar-day6\enterprise-contract-agent

2. Start Docker services:
   docker-compose up -d

3. Access the dashboards:
   • Prometheus:  http://localhost:9090
   • Grafana:     http://localhost:3000 (admin/admin)
   • Jaeger:      http://localhost:16686 (traces)

4. In Grafana:
   - Add Prometheus datasource (http://prometheus:9090)
   - Import dashboard from dashboards/contract_agent.json

5. Run the agent and see metrics appear in real-time!


✅ Instructions displayed above


## Key Takeaways

### **What We Learned:**

1. **Structured Logging**
   - JSON format for machine parsing
   - Context management for request tracking
   - Rich metadata for debugging

2. **Prometheus Metrics**
   - Counter, Gauge, Histogram types
   - Custom business metrics (contract types, tokens)
   - Performance tracking (duration, throughput)

3. **OpenTelemetry Tracing**
   - Distributed traces across services
   - Span hierarchy for nested operations
   - Custom attributes and events

4. **Integration Patterns**
   - Instrumenting LangGraph nodes
   - Error handling with observability
   - Context propagation

### **Production Benefits:**

- **Faster debugging** - Find issues in minutes, not hours
- **Cost optimization** - Track token usage per user/contract type
- **SLA compliance** - Monitor and alert on performance
- **Capacity planning** - Predict scaling needs
- **User experience** - Identify slow operations
